**I recommend you run the first code cell of this notebook immediately, to start provisioning drake on the cloud machine, then you can leave this window open as you [read the textbook](manipulation.csail.mit.edu/pose.html).**

# Notebook setup

The following cell will:
- on Colab (only), install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.  This will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  If you navigate between notebooks using Colab's "File->Open" menu, then you can avoid provisioning a separate machine for each notebook.
- launch a server for our 3D visualizer (MeshCat) that will be used for the remainder of this notebook.

You will need to rerun this cell if you restart the kernel, but it should be fast because the machine will already have drake installed.

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

if 'google.colab' in sys.modules and importlib.util.find_spec('manipulation') is None:
    urlretrieve(f"http://manipulation.csail.mit.edu/scripts/setup/setup_manipulation_colab.py",
                "setup_manipulation_colab.py")
    from setup_manipulation_colab import setup_manipulation
    setup_manipulation(manipulation_sha='master', drake_version='20200917', drake_build='nightly')

# Determine if this notebook is currently running as a notebook or a unit test.
from IPython import get_ipython
running_as_notebook = get_ipython() and hasattr(get_ipython(), 'kernel')

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
server_args = []
if 'google.colab' in sys.modules:
    server_args = ['--ngrok_http_tunnel']
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

# Teleop Example

In [ ]:
import numpy as np
from ipywidgets import ToggleButton, ToggleButtons
from pydrake.all import ( set_log_level,
    ConnectMeshcatVisualizer, DiagramBuilder, 
    DifferentialInverseKinematicsParameters, 
    DifferentialInverseKinematicsIntegrator,
    RigidTransform, RotationMatrix, Simulator
)
from pydrake.systems.jupyter_widgets import PoseSliders, WidgetSystem
from pydrake.examples.manipulation_station import ManipulationStation, CreateClutterClearingYcbObjectList


set_log_level("warn")
builder = DiagramBuilder()

station = builder.AddSystem(ManipulationStation())

station.SetupClutterClearingStation()
#ycb_objects = CreateClutterClearingYcbObjectList()
#for model_file, X_WObject in ycb_objects:
#    station.AddManipulandFromFile(model_file, X_WObject)
station.AddManipulandFromFile(
    "drake/examples/manipulation_station/models/"
    + "061_foam_brick.sdf",
    RigidTransform(RotationMatrix.Identity(), [0, -0.6, 0.2]))
station.Finalize()

visualizer = ConnectMeshcatVisualizer(
    builder, 
    station.get_scene_graph(), 
    station.GetOutputPort("pose_bundle"),
    zmq_url=zmq_url,
    open_browser=False,
    server_args=server_args)
#    jupyter_comms=True)

robot = station.get_controller_plant()
params = DifferentialInverseKinematicsParameters(robot.num_positions(),
                                                  robot.num_velocities())

time_step = 0.005
params.set_timestep(time_step)
# True velocity limits for the IIWA14 (in rad, rounded down to the first
# decimal)
iiwa14_velocity_limits = np.array([1.4, 1.4, 1.7, 1.3, 2.2, 2.3, 2.3])
params.set_joint_velocity_limits((-iiwa14_velocity_limits,
                                  iiwa14_velocity_limits))
differential_ik = builder.AddSystem(DifferentialInverseKinematicsIntegrator(
    robot, robot.GetFrameByName("iiwa_link_7"), time_step, params))
builder.Connect(differential_ik.get_output_port(),
                station.GetInputPort("iiwa_position"))

teleop = builder.AddSystem(PoseSliders(
    min_range = PoseSliders.MinRange(roll=0, pitch=-0.5, yaw=-np.pi, 
                                     x=-0.6, y=-0.8, z=0.0),
    max_range = PoseSliders.MaxRange(roll=2*np.pi, pitch=np.pi, yaw=np.pi,
                                     x=0.8, y=0.3, z=1.1)
))
builder.Connect(teleop.get_output_port(0), 
                differential_ik.get_input_port())
wsg_buttons = ToggleButtons(value=0.107, description="SchunkWsg", 
                            options=[('Open', 0.107), ('Close', 0.002)])
wsg_teleop = builder.AddSystem(WidgetSystem([wsg_buttons]))
builder.Connect(wsg_teleop.get_output_port(0),
                station.GetInputPort("wsg_position"))

diagram = builder.Build()
simulator = Simulator(diagram)
context = simulator.get_mutable_context()

station_context = station.GetMyMutableContextFromRoot(context)

q0 = station.GetOutputPort("iiwa_position_measured").Eval(
    station_context)
differential_ik.get_mutable_parameters().set_nominal_joint_position(q0)
diff_ik_context = differential_ik.GetMyMutableContextFromRoot(context)
differential_ik.SetPositions(diff_ik_context, q0)
teleop.SetPose(differential_ik.ForwardKinematics(diff_ik_context))

if running_as_notebook:  # Then we're not just running as a test on CI.
    simulator.set_target_realtime_rate(1.0)

    stop_button = ToggleButton(value=False, description='Stop Simulation')
    display(stop_button)
    while not stop_button.value:
        simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)
    stop_button.value = False
  
else:
    simulator.AdvanceTo(0.1)

# Visualizing Jacobian-based control

In [ ]:
import numpy as np
from functools import partial
from IPython.display import display
from ipywidgets import FloatSlider, Textarea, Layout

from pydrake.all import (DiagramBuilder, AddMultibodyPlantSceneGraph, RigidTransform, ConnectMeshcatVisualizer,
                         JacobianWrtVariable, Parser, FixedOffsetFrame,
                         PiecewisePolynomial, MathematicalProgram, LinearConstraint, Solve)
from manipulation.utils import FindResource


# TODO(russt): upstream this to drake
import meshcat.geometry as g
import meshcat.transformations as tf

def plot_surface(meshcat_viz, X, Y, Z, color=0xdd9999, wireframe=False):
    (rows, cols) = Z.shape

    vertices = np.empty((rows*cols,3),dtype=np.float32)
    vertices[:,0] = X.reshape((-1))
    vertices[:,1] = Y.reshape((-1))
    vertices[:,2] = Z.reshape((-1))

    # Vectorized faces code from https://stackoverflow.com/questions/44934631/making-grid-triangular-mesh-quickly-with-numpy
    faces = np.empty((cols-1,rows-1,2,3),dtype=np.uint32)
    r = np.arange(rows*cols).reshape(cols,rows)
    faces[:,:, 0,0] = r[:-1,:-1]
    faces[:,:, 1,0] = r[:-1,1:]
    faces[:,:, 0,1] = r[:-1,1:]
    faces[:,:, 1,1] = r[1:,1:]
    faces[:,:, :,2] = r[1:,:-1,None]
    faces.shape =(-1,3)
    meshcat_viz.set_object(
        g.TriangularMeshGeometry(vertices, faces), 
        g.MeshLambertMaterial(color=color, wireframe=wireframe))

def plot_mathematical_program(meshcat, prog, X, Y, result=None):
    assert prog.num_vars() == 2
    assert X.size == Y.size

    N = X.size
    values = np.vstack((X.reshape(-1), Y.reshape(-1)))
    costs = prog.GetAllCosts()

    # Vectorized multiply for the quadratic form.
    # Z = (D*np.matmul(Q,D)).sum(0).reshape(nx, ny)

    if costs:
        Z = prog.EvalBindingVectorized(costs[0], values)
        for b in costs[1:]:
            Z = Z + prog.EvalBindingVectorized(b, values)

    cv = meshcat["constraint"]                
    for binding in prog.GetAllConstraints():
        c = binding.evaluator()
        var_indices = [int(prog.decision_variable_index()[v.get_id()]) for v in binding.variables()]
        satisfied = np.array(c.CheckSatisfiedVectorized(values[var_indices, :], 0.001)).reshape(1, -1)
        if costs:
            Z[~satisfied] = np.nan

        # Special case linear constraints
        if False: #isinstance(c, LinearConstraint):
            # TODO: take these as (optional) arguments to avoid computing them inefficiently.
            xmin = np.min(X.reshape(-1))
            xmax = np.max(X.reshape(-1))
            ymin = np.min(Y.reshape(-1))
            ymax = np.max(Y.reshape(-1))
            A = c.A()
            lower = c.lower_bound()
            upper = c.upper_bound()
            # find line / box intersections
            # https://gist.github.com/ChickenProp/3194723
        else:
            v = cv[str(binding)]
            Zc = np.zeros(Z.shape)
            Zc[satisfied] = np.nan
            plot_surface(v, X, Y, Zc.reshape((X.shape[1], X.shape[0])), color=0x9999dd)

    if costs:
        plot_surface(meshcat["objective"], X, Y, Z.reshape(X.shape[1], X.shape[0]), wireframe=True)

    if result:
        v = meshcat["solution"]
        v.set_object(g.Sphere(0.1), g.MeshLambertMaterial(color=0x99ff99))
        x_solution = result.get_x_val()
        v.set_transform(tf.translation_matrix([x_solution[0], x_solution[1], result.get_optimal_cost()]))


# This one is specific to this notebook, but I'm putting it in the header to make it less distracting.
def Visualizer(MakeMathematicalProgram):
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
    twolink = Parser(plant, scene_graph).AddModelFromFile(
        FindResource("models/double_pendulum.urdf"))
    hand = plant.AddFrame(frame=FixedOffsetFrame(
        name="hand", P=plant.GetFrameByName("lower_arm"),
        X_PF=RigidTransform([0, 0, -1])))
    plant.Finalize()

    meshcat = ConnectMeshcatVisualizer(builder,
                                  scene_graph,
                                  zmq_url=zmq_url)
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    meshcat.vis.delete()
    #meshcat.vis["/Background"].set_property('visible',False)
    meshcat.vis["/Background"].set_property('top_color', [0, 0, 0])
    meshcat.vis["/Background"].set_property('bottom_color', [0, 0, 0])
    meshcat.vis["/Grid"].set_property('visible',False)
    meshcat.load()

    jacobian = Textarea(value="", description="J_G: ", layout={'width':'200pm','height':'50px'}, style={'description_width':'initial'})
    display(jacobian)

    X, Y = np.meshgrid(np.linspace(-5, 5, 35), np.linspace(-5, 5, 31))

    def visualize(q, v_Gdesired=[1.0, 0.0], t=None):
        if t:
            context.SetTime(t)
        plant.SetPositions(plant_context, q)
        diagram.Publish(context)

        J_G = plant.CalcJacobianTranslationalVelocity(plant_context, JacobianWrtVariable.kQDot, hand, [0,0,0], plant.world_frame(), plant.world_frame())
        J_G = J_G[[0,2],:]  # Ignore Y.
        jacobian.value = np.array2string(J_G, formatter={'float': lambda x: "{:5.2f}".format(x)})

        prog = MakeMathematicalProgram(q, J_G, v_Gdesired)
        result = Solve(prog)
        v = meshcat.vis["QP"]
        plot_mathematical_program(v, prog, X, Y, result=result)
        # TODO: Add set_object to meshcat.Animation
        if False: # meshcat._is_recording:
            with meshcat._animation.at_frame(
                    v, meshcat._recording_frame_num) as m:
                plot_mathematical_program(m, prog, X, Y, result=result)

    return visualize, meshcat

def MakeMathematicalProgram(q, J_G, v_Gdesired):
    prog = MathematicalProgram()
    v = prog.NewContinuousVariables(2, 'v')
    v_max = 3.0 

    error = J_G.dot(v) - v_Gdesired
    prog.AddCost(error.dot(error))
    prog.AddBoundingBoxConstraint(-v_max, v_max, v)

    return prog

visualize, meshcat = Visualizer(MakeMathematicalProgram)

q = [-np.pi/2.0+0.5, -1.0]
v_Gdesired = [0.5, 0.]
visualize(q, v_Gdesired)

In [ ]:
import time

visualize, meshcat = Visualizer(MakeMathematicalProgram)

v_Gdesired = [1.0, 0.0]
T = 2.
q = PiecewisePolynomial.FirstOrderHold(
    [0, T, 2*T], np.array([[-np.pi / 2.0 + 1., -np.pi / 2.0 - 1., -np.pi / 2.0 + 1.], 
                           [-2., 2., -2]]))

nx = 35
ny = 31
X, Y = np.meshgrid(np.linspace(-5, 5, nx), np.linspace(-5, 5, ny))
D = np.vstack((X.reshape(1,-1), Y.reshape(1,-1)))
for i in range(2):
    for t in np.linspace(0, 2*T, num=100):
        visualize(q.value(t), v_Gdesired, t=t)
        time.sleep(0.05)